In [1]:
import json
import os
import math
import librosa
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd
import tensorflow as tf

C:\Users\Alexander\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Alexander\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Alexander\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Alexander\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: Futu

In [2]:
#Variable definitions
#signal-Signal S
#frame_size -divided frames
#hop_length-size intersection between frames
#P- given parameter
#defined function- Input = signal; Output = t_new point   (Step 1-4 in the Paper)
def amplitude_envelope(signal,sample_duration, frame_size=1024, hop_length=512,P=0.35):
    """Calculate the amplitude envelope of a signal with a given frame size nad hop length."""
    amplitude_envelope = []  #Define amplitude envolope list
    index_signal_amplitude_envelope=[]  #Define index for amplitude envelope
    # calculate amplitude envelope for each frame
    for i in range(0, len(signal), hop_length): 
        amplitude_envelope_current_frame = max(signal[i:i+frame_size]) 
        amplitude_envelope.append(amplitude_envelope_current_frame)
        #append index for each amplitude envelope obtained in each frame
        index_signal_amplitude_envelope.append(i+signal[i:i+frame_size].tolist().index(max(signal[i:i+frame_size].tolist())))
    
    ae_signal=np.array(amplitude_envelope)                        #list of amplitude envelope
    t=np.array(index_signal_amplitude_envelope)*sample_duration   #corresponding time for amplitude envelope
    t_max=t[ae_signal.tolist().index(max(ae_signal))]       #t_new=time for max amplitude - 0.35   #0.35 is a parameter
    
    if t_max<=P:
        t_new=0
    else:
        t_new=t_max-P
    return t_new

In [3]:
def save_mfcc(dataset_path, n_fft, hop_length):
    """Extracts MFCCs from music dataset and saves them into a json file along witgh genre labels.
        :param dataset_path (str): Path to dataset
        :param json_path (str): Path to json file used to save MFCCs
        :param num_mfcc (int): Number of coefficients to extract
        :param n_fft (int): Interval we consider to apply FFT. Measured in # of samples
        :param hop_length (int): Sliding window for FFT. Measured in # of samples
        :param: num_segments (int): Number of segments we want to divide sample tracks into
        :return:
        """

    # dictionary to store mapping, labels, and MFCCs
    data = {
        "labels": [],  # 1,2,3,4
        "mfcc": [],    # STFTs...,mels,.....MFCCs
        "region":[]   #regions....,...,
    }
   
    time_o=[]
 
    j=0
    a=0
    p=0
    #Section 4.1.1 Signal length module
    # loop through all sub-folder
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        
        if dirpath is not dataset_path:
            # save DIGIT label  in the mapping
            region_label = dirpath.split(" ")[0].split("\\")[1]
            print("\nProcessing: {}".format(region_label))
            
            # process all audio files 
            
            for f in filenames:
                # save DIGIT label  in the mapping
                digit_label = f.split("D")[1].split(".")[0]
                #print(f)
                #print(digit_label)
                #speaker_label = f.split("_")[1]
                #print(speaker_label)
                
                # Save file path with source dir path
                file_path = os.path.join(dirpath, f)
                
                # input file wav file to obtain signal and new sample rate
                signal, sr = librosa.load(file_path, sr=SAMPLE_RATE) 
                                
                # New output file name
                trimmed_path="recordings-github\\"+dirpath.split("\\")[1]+"\\"+dirpath.split("\\")[2]+"\\"+f
                
                # Display
                print(trimmed_path)
                # Calculate duration in seconds for the input Signal
                dur=len(signal)/SAMPLE_RATE
                sample_duration = 1 / sr
                
                if dur>=0.74:
                   # print(dur)
                    p+=1
                
                # Step 1-5
                if len(signal)>= num_expected_samples:     #Step 5 checking for trimming
                    offset=amplitude_envelope(signal,sample_duration)   #Step 1-4
                    trim_signal, sr = librosa.load(file_path,sr=SAMPLE_RATE,offset=offset,duration=DURATION) #trimming
                    padded_signal=trim_signal
                    if len(trim_signal)<num_expected_samples:
                        num_missing_samples = num_expected_samples - len(trim_signal)
                        padded_signal = np.pad(trim_signal,
                                              (0, num_missing_samples),
                                              mode="constant")      #Padding
                        a+=1
                elif len(signal) < num_expected_samples:     #Step 5 checking for padding
                    num_missing_samples = num_expected_samples - len(signal)
                    padded_signal = np.pad(signal,
                                          (0, num_missing_samples),
                                          mode="constant")    #Padding
                    
                #librosa.output.write_wav(trimmed_path, padded_signal, sr)
                
                # Section 4.1.2 Feature extraction module
                # Input new signal ; Output MFCCs
                # New signal=padded_signal; sr=sample rate; n_mfcc=no of mfcc coefficients; n_fft=interval for fft; hop_length=sliding window
                mfcc = librosa.feature.mfcc(padded_signal, sr, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                
                # Display mfcc dimension 
                print(mfcc.shape)
                
                # normalizing
                S_min=np.min(mfcc)
                S_max=np.max(mfcc)
                X_norm_mfcc=(mfcc - S_min)/(S_max - S_min)
                
                # append normalized mfccs features, labels, and regions labels to each data containers
                data["mfcc"].append(X_norm_mfcc.T.tolist())
                data["labels"].append(digit_label)
                data["region"].append(region_label)
              
                j+=1

 
    # save MFCCs and its labels to json file
    with open(JSON_PATH, "w") as fp:
        json.dump(data, fp, indent=4)

In [4]:
# Defined parameters
DATASET_PATH = "recordings"   #"data"
JSON_PATH =  "MNIST-digit-gu-mfcc-norm-16.json"  #"data_10.json" MNIST-digit-sw-mfcc-norm-16.json
SAMPLE_RATE = 22050 #22050 #   8000
n_fft=512       #511  358
hop_length=256   #123  296
DURATION=0.74   #2   0.74
num_expected_samples=int(DURATION*SAMPLE_RATE)
num_mfcc=16

In [5]:
if __name__ == "__main__":
    save_mfcc(DATASET_PATH,n_fft, hop_length)   # run save_mfcc function


Processing: R1

Processing: R1
recordings-github\R1 - Central Zone\S1\R1S1T1D0.wav
(16, 64)
recordings-github\R1 - Central Zone\S1\R1S1T1D1.wav
(16, 64)
recordings-github\R1 - Central Zone\S1\R1S1T1D2.wav
(16, 64)
recordings-github\R1 - Central Zone\S1\R1S1T1D3.wav
(16, 64)
recordings-github\R1 - Central Zone\S1\R1S1T1D4.wav
(16, 64)
recordings-github\R1 - Central Zone\S1\R1S1T1D5.wav
(16, 64)
recordings-github\R1 - Central Zone\S1\R1S1T1D6.wav
(16, 64)
recordings-github\R1 - Central Zone\S1\R1S1T1D7.wav
(16, 64)
recordings-github\R1 - Central Zone\S1\R1S1T1D8.wav
(16, 64)
recordings-github\R1 - Central Zone\S1\R1S1T1D9.wav
(16, 64)
recordings-github\R1 - Central Zone\S1\R1S1T2D0.wav
(16, 64)
recordings-github\R1 - Central Zone\S1\R1S1T2D1.wav
(16, 64)
recordings-github\R1 - Central Zone\S1\R1S1T2D2.wav
(16, 64)
recordings-github\R1 - Central Zone\S1\R1S1T2D3.wav
(16, 64)
recordings-github\R1 - Central Zone\S1\R1S1T2D4.wav
(16, 64)
recordings-github\R1 - Central Zone\S1\R1S1T2D5.wav
(

(16, 64)
recordings-github\R1 - Central Zone\S3\R1S3T10D4.wav
(16, 64)
recordings-github\R1 - Central Zone\S3\R1S3T10D5.wav
(16, 64)
recordings-github\R1 - Central Zone\S3\R1S3T10D6.wav
(16, 64)
recordings-github\R1 - Central Zone\S3\R1S3T10D7.wav
(16, 64)
recordings-github\R1 - Central Zone\S3\R1S3T10D8.wav
(16, 64)
recordings-github\R1 - Central Zone\S3\R1S3T10D9.wav
(16, 64)
recordings-github\R1 - Central Zone\S3\R1S3T1D0.wav
(16, 64)
recordings-github\R1 - Central Zone\S3\R1S3T1D1.wav
(16, 64)
recordings-github\R1 - Central Zone\S3\R1S3T1D2.wav
(16, 64)
recordings-github\R1 - Central Zone\S3\R1S3T1D3.wav
(16, 64)
recordings-github\R1 - Central Zone\S3\R1S3T1D4.wav
(16, 64)
recordings-github\R1 - Central Zone\S3\R1S3T1D5.wav
(16, 64)
recordings-github\R1 - Central Zone\S3\R1S3T1D6.wav
(16, 64)
recordings-github\R1 - Central Zone\S3\R1S3T1D7.wav
(16, 64)
recordings-github\R1 - Central Zone\S3\R1S3T1D8.wav
(16, 64)
recordings-github\R1 - Central Zone\S3\R1S3T1D9.wav
(16, 64)
recording

(16, 64)
recordings-github\R1 - Central Zone\S4\R1S4T3D9.wav
(16, 64)
recordings-github\R1 - Central Zone\S4\R1S4T4D0.wav
(16, 64)
recordings-github\R1 - Central Zone\S4\R1S4T4D1.wav
(16, 64)
recordings-github\R1 - Central Zone\S4\R1S4T4D2.wav
(16, 64)
recordings-github\R1 - Central Zone\S4\R1S4T4D3.wav
(16, 64)
recordings-github\R1 - Central Zone\S4\R1S4T4D4.wav
(16, 64)
recordings-github\R1 - Central Zone\S4\R1S4T4D5.wav
(16, 64)
recordings-github\R1 - Central Zone\S4\R1S4T4D6.wav
(16, 64)
recordings-github\R1 - Central Zone\S4\R1S4T4D7.wav
(16, 64)
recordings-github\R1 - Central Zone\S4\R1S4T4D8.wav
(16, 64)
recordings-github\R1 - Central Zone\S4\R1S4T4D9.wav
(16, 64)
recordings-github\R1 - Central Zone\S4\R1S4T5D0.wav
(16, 64)
recordings-github\R1 - Central Zone\S4\R1S4T5D1.wav
(16, 64)
recordings-github\R1 - Central Zone\S4\R1S4T5D2.wav
(16, 64)
recordings-github\R1 - Central Zone\S4\R1S4T5D3.wav
(16, 64)
recordings-github\R1 - Central Zone\S4\R1S4T5D4.wav
(16, 64)
recordings-gith

recordings-github\R1 - Central Zone\S5\R1S5T7D4.wav
(16, 64)
recordings-github\R1 - Central Zone\S5\R1S5T7D5.wav
(16, 64)
recordings-github\R1 - Central Zone\S5\R1S5T7D6.wav
(16, 64)
recordings-github\R1 - Central Zone\S5\R1S5T7D7.wav
(16, 64)
recordings-github\R1 - Central Zone\S5\R1S5T7D8.wav
(16, 64)
recordings-github\R1 - Central Zone\S5\R1S5T7D9.wav
(16, 64)
recordings-github\R1 - Central Zone\S5\R1S5T8D0.wav
(16, 64)
recordings-github\R1 - Central Zone\S5\R1S5T8D1.wav
(16, 64)
recordings-github\R1 - Central Zone\S5\R1S5T8D2.wav
(16, 64)
recordings-github\R1 - Central Zone\S5\R1S5T8D3.wav
(16, 64)
recordings-github\R1 - Central Zone\S5\R1S5T8D4.wav
(16, 64)
recordings-github\R1 - Central Zone\S5\R1S5T8D5.wav
(16, 64)
recordings-github\R1 - Central Zone\S5\R1S5T8D6.wav
(16, 64)
recordings-github\R1 - Central Zone\S5\R1S5T8D7.wav
(16, 64)
recordings-github\R1 - Central Zone\S5\R1S5T8D8.wav
(16, 64)
recordings-github\R1 - Central Zone\S5\R1S5T8D9.wav
(16, 64)
recordings-github\R1 - C

recordings-github\R2 - North Zone\S2\R2S2T1D3.wav
(16, 64)
recordings-github\R2 - North Zone\S2\R2S2T1D4.wav
(16, 64)
recordings-github\R2 - North Zone\S2\R2S2T1D5.wav
(16, 64)
recordings-github\R2 - North Zone\S2\R2S2T1D6.wav
(16, 64)
recordings-github\R2 - North Zone\S2\R2S2T1D7.wav
(16, 64)
recordings-github\R2 - North Zone\S2\R2S2T1D8.wav
(16, 64)
recordings-github\R2 - North Zone\S2\R2S2T1D9.wav
(16, 64)
recordings-github\R2 - North Zone\S2\R2S2T2D0.wav
(16, 64)
recordings-github\R2 - North Zone\S2\R2S2T2D1.wav
(16, 64)
recordings-github\R2 - North Zone\S2\R2S2T2D2.wav
(16, 64)
recordings-github\R2 - North Zone\S2\R2S2T2D3.wav
(16, 64)
recordings-github\R2 - North Zone\S2\R2S2T2D4.wav
(16, 64)
recordings-github\R2 - North Zone\S2\R2S2T2D5.wav
(16, 64)
recordings-github\R2 - North Zone\S2\R2S2T2D6.wav
(16, 64)
recordings-github\R2 - North Zone\S2\R2S2T2D7.wav
(16, 64)
recordings-github\R2 - North Zone\S2\R2S2T2D8.wav
(16, 64)
recordings-github\R2 - North Zone\S2\R2S2T2D9.wav
(16, 6

...

recordings-github\R3 - South Zone\S4\R3S4T1D1.wav
(16, 64)
recordings-github\R3 - South Zone\S4\R3S4T1D2.wav
(16, 64)
recordings-github\R3 - South Zone\S4\R3S4T1D3.wav
(16, 64)
recordings-github\R3 - South Zone\S4\R3S4T1D4.wav
(16, 64)
recordings-github\R3 - South Zone\S4\R3S4T1D5.wav
(16, 64)
recordings-github\R3 - South Zone\S4\R3S4T1D6.wav
(16, 64)
recordings-github\R3 - South Zone\S4\R3S4T1D7.wav
(16, 64)
recordings-github\R3 - South Zone\S4\R3S4T1D8.wav
(16, 64)
recordings-github\R3 - South Zone\S4\R3S4T1D9.wav
(16, 64)
recordings-github\R3 - South Zone\S4\R3S4T2D0.wav
(16, 64)
recordings-github\R3 - South Zone\S4\R3S4T2D1.wav
(16, 64)
recordings-github\R3 - South Zone\S4\R3S4T2D2.wav
(16, 64)
recordings-github\R3 - South Zone\S4\R3S4T2D3.wav
(16, 64)
recordings-github\R3 - South Zone\S4\R3S4T2D4.wav
(16, 64)
recordings-github\R3 - South Zone\S4\R3S4T2D5.wav
(16, 64)
recordings-github\R3 - South Zone\S4\R3S4T2D6.wav
(16, 64)
recordings-github\R3 - South Zone\S4\R3S4T2D7.wav
(16, 6

recordings-github\R4 - Saurashtra\S1\R4S1T3D9.wav
(16, 64)
recordings-github\R4 - Saurashtra\S1\R4S1T4D0.wav
(16, 64)
recordings-github\R4 - Saurashtra\S1\R4S1T4D1.wav
(16, 64)
recordings-github\R4 - Saurashtra\S1\R4S1T4D2.wav
(16, 64)
recordings-github\R4 - Saurashtra\S1\R4S1T4D3.wav
(16, 64)
recordings-github\R4 - Saurashtra\S1\R4S1T4D4.wav
(16, 64)
recordings-github\R4 - Saurashtra\S1\R4S1T4D5.wav
(16, 64)
recordings-github\R4 - Saurashtra\S1\R4S1T4D6.wav
(16, 64)
recordings-github\R4 - Saurashtra\S1\R4S1T4D7.wav
(16, 64)
recordings-github\R4 - Saurashtra\S1\R4S1T4D8.wav
(16, 64)
recordings-github\R4 - Saurashtra\S1\R4S1T4D9.wav
(16, 64)
recordings-github\R4 - Saurashtra\S1\R4S1T5D0.wav
(16, 64)
recordings-github\R4 - Saurashtra\S1\R4S1T5D1.wav
(16, 64)
recordings-github\R4 - Saurashtra\S1\R4S1T5D2.wav
(16, 64)
recordings-github\R4 - Saurashtra\S1\R4S1T5D3.wav
(16, 64)
recordings-github\R4 - Saurashtra\S1\R4S1T5D4.wav
(16, 64)
recordings-github\R4 - Saurashtra\S1\R4S1T5D5.wav
(16, 6

recordings-github\R4 - Saurashtra\S2\R4S2T7D8.wav
(16, 64)
recordings-github\R4 - Saurashtra\S2\R4S2T7D9.wav
(16, 64)
recordings-github\R4 - Saurashtra\S2\R4S2T8D0.wav
(16, 64)
recordings-github\R4 - Saurashtra\S2\R4S2T8D1.wav
(16, 64)
recordings-github\R4 - Saurashtra\S2\R4S2T8D2.wav
(16, 64)
recordings-github\R4 - Saurashtra\S2\R4S2T8D3.wav
(16, 64)
recordings-github\R4 - Saurashtra\S2\R4S2T8D4.wav
(16, 64)
recordings-github\R4 - Saurashtra\S2\R4S2T8D5.wav
(16, 64)
recordings-github\R4 - Saurashtra\S2\R4S2T8D6.wav
(16, 64)
recordings-github\R4 - Saurashtra\S2\R4S2T8D7.wav
(16, 64)
recordings-github\R4 - Saurashtra\S2\R4S2T8D8.wav
(16, 64)
recordings-github\R4 - Saurashtra\S2\R4S2T8D9.wav
(16, 64)
recordings-github\R4 - Saurashtra\S2\R4S2T9D0.wav
(16, 64)
recordings-github\R4 - Saurashtra\S2\R4S2T9D1.wav
(16, 64)
recordings-github\R4 - Saurashtra\S2\R4S2T9D2.wav
(16, 64)
recordings-github\R4 - Saurashtra\S2\R4S2T9D3.wav
(16, 64)
recordings-github\R4 - Saurashtra\S2\R4S2T9D4.wav
(16, 6

recordings-github\R4 - Saurashtra\S4\R4S4T1D6.wav
(16, 64)
recordings-github\R4 - Saurashtra\S4\R4S4T1D7.wav
(16, 64)
recordings-github\R4 - Saurashtra\S4\R4S4T1D8.wav
(16, 64)
recordings-github\R4 - Saurashtra\S4\R4S4T1D9.wav
(16, 64)
recordings-github\R4 - Saurashtra\S4\R4S4T2D0.wav
(16, 64)
recordings-github\R4 - Saurashtra\S4\R4S4T2D1.wav
(16, 64)
recordings-github\R4 - Saurashtra\S4\R4S4T2D2.wav
(16, 64)
recordings-github\R4 - Saurashtra\S4\R4S4T2D3.wav
(16, 64)
recordings-github\R4 - Saurashtra\S4\R4S4T2D4.wav
(16, 64)
recordings-github\R4 - Saurashtra\S4\R4S4T2D5.wav
(16, 64)
recordings-github\R4 - Saurashtra\S4\R4S4T2D6.wav
(16, 64)
recordings-github\R4 - Saurashtra\S4\R4S4T2D7.wav
(16, 64)
recordings-github\R4 - Saurashtra\S4\R4S4T2D8.wav
(16, 64)
recordings-github\R4 - Saurashtra\S4\R4S4T2D9.wav
(16, 64)
recordings-github\R4 - Saurashtra\S4\R4S4T3D0.wav
(16, 64)
recordings-github\R4 - Saurashtra\S4\R4S4T3D1.wav
(16, 64)
recordings-github\R4 - Saurashtra\S4\R4S4T3D2.wav
(16, 6

recordings-github\R4 - Saurashtra\S5\R4S5T5D7.wav
(16, 64)
recordings-github\R4 - Saurashtra\S5\R4S5T5D8.wav
(16, 64)
recordings-github\R4 - Saurashtra\S5\R4S5T5D9.wav
(16, 64)
recordings-github\R4 - Saurashtra\S5\R4S5T6D0.wav
(16, 64)
recordings-github\R4 - Saurashtra\S5\R4S5T6D1.wav
(16, 64)
recordings-github\R4 - Saurashtra\S5\R4S5T6D2.wav
(16, 64)
recordings-github\R4 - Saurashtra\S5\R4S5T6D3.wav
(16, 64)
recordings-github\R4 - Saurashtra\S5\R4S5T6D4.wav
(16, 64)
recordings-github\R4 - Saurashtra\S5\R4S5T6D5.wav
(16, 64)
recordings-github\R4 - Saurashtra\S5\R4S5T6D6.wav
(16, 64)
recordings-github\R4 - Saurashtra\S5\R4S5T6D7.wav
(16, 64)
recordings-github\R4 - Saurashtra\S5\R4S5T6D8.wav
(16, 64)
recordings-github\R4 - Saurashtra\S5\R4S5T6D9.wav
(16, 64)
recordings-github\R4 - Saurashtra\S5\R4S5T7D0.wav
(16, 64)
recordings-github\R4 - Saurashtra\S5\R4S5T7D1.wav
(16, 64)
recordings-github\R4 - Saurashtra\S5\R4S5T7D2.wav
(16, 64)
recordings-github\R4 - Saurashtra\S5\R4S5T7D3.wav
(16, 6